In [119]:
import numpy as np
from scipy import io as sio
import pandas as pd
from shutil import copyfile

In [35]:
raw = sio.loadmat('./wiki/wiki.mat')

In [36]:
type(raw)

dict

In [37]:
raw.keys()

dict_keys(['__header__', '__version__', '__globals__', 'wiki'])

In [38]:
raw = raw['wiki']

In [39]:
type(raw)

numpy.ndarray

In [40]:
raw.shape

(1, 1)

In [41]:
raw = raw[0][0]

In [42]:
type(raw)

numpy.void

In [43]:
raw.shape

()

In [44]:
len(raw)

8

In [45]:
data = []
for i in raw:
    data.append(i)
data = np.array(data)

In [46]:
data.shape

(8, 1, 62328)

In [47]:
data = data.squeeze()

In [48]:
data.shape

(8, 62328)

In [49]:
data = data.T
data.shape

(62328, 8)

In [84]:
df = pd.DataFrame(data)
df.columns = ['dob', 'photo_taken', 'full_path', 'gender', 'name', 'face_location', 'face_score', 'second_face_score']

In [85]:
df = df[df['face_score'] != float('-inf')]

In [86]:
df = df.dropna(subset=['gender'])

In [87]:
df = df[pd.isnull(df['second_face_score'])]

In [88]:
from datetime import datetime, timedelta
def mat_date_convert(mat_date):
    return (timedelta(days=mat_date -366)+datetime(1,1,1)).year

In [90]:
df['age'] = df['photo_taken'] - df['dob'].apply(mat_date_convert)

In [95]:
proccesed_data = df[['full_path', 'gender', 'age']]

In [122]:
# To run again uncomment

# female_path = './proccessed_data/gender/female/'
# male_path = './proccessed_data/gender/male/'
# wiki_path = './wiki_crop/'
# for i, row in proccesed_data.iterrows():
#     path = row['full_path'][0]
#     name = path.replace('/', '_')
#     old_path = wiki_path + path
#     age = row['age']
#     gender = row['gender']
#     if gender == 0: # female
#         new_path = female_path + name
#     else: # male
#         new_path = male_path + name
#     copyfile(old_path, new_path)